In [1]:
import keras
import tensorflow as tf
import keras.backend as K

from keras.layers import Conv2D, SeparableConv2D, Conv2DTranspose, add, ReLU, Dropout, Reshape, Permute
from keras.layers import Activation, Dense, Add
from keras.models import Model

Using TensorFlow backend.
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hyunkim/.pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [2]:
from models import matting_net

model = matting_net()

model.load_weights("./trained_models/portrait_seg_matting_256_191017_val_loss_0.0404_val_acc_0.9725_focal_1312.8737.h5")

In [3]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 4)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  224         lambda_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 256, 256, 8)  0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [4]:
# converting

In [5]:
outPath = "./tflite/model_android_191017_linear.tflite"

input_names = [node.op.name for node in model.inputs]
output_names = [node.op.name for node in model.outputs]
print(input_names)
print(output_names)

sess = K.get_session()
converter = tf.lite.TFLiteConverter.from_session(sess, model.inputs, model.outputs)

tflite_model = converter.convert()
open(outPath, "wb").write(tflite_model)

['input_1']
['output/Sigmoid']
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 166 variables.
INFO:tensorflow:Converted 166 variables to const ops.


3740276

In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 4)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 8)  224         lambda_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 256, 256, 8)  0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [7]:
import numpy as np 
import matplotlib.pyplot as plt
random_img = np.random.rand(1,256,256,4)
plt.imshow(model.predict(random_img).reshape(256, 256))

In [8]:
model.predict(random_img)

array([[[[1.1102557e-03],
         [3.2955408e-04],
         [1.2126565e-04],
         ...,
         [3.3274591e-03],
         [1.4655292e-03],
         [2.2450686e-03]],

        [[1.4978647e-04],
         [8.5294247e-05],
         [2.4527311e-05],
         ...,
         [1.8713474e-03],
         [1.1292696e-03],
         [2.5552511e-03]],

        [[3.7103891e-05],
         [4.5299530e-06],
         [1.5199184e-06],
         ...,
         [1.2130141e-03],
         [9.5835328e-04],
         [1.4164746e-03]],

        ...,

        [[8.2805753e-04],
         [6.6119432e-04],
         [6.1988831e-04],
         ...,
         [9.9956852e-01],
         [9.9839634e-01],
         [8.2523417e-01]],

        [[1.6538501e-03],
         [1.2415051e-03],
         [1.4998019e-03],
         ...,
         [9.9949396e-01],
         [9.9559224e-01],
         [8.5188818e-01]],

        [[3.3655167e-03],
         [3.7128329e-03],
         [2.2377968e-03],
         ...,
         [9.7872114e-01],
        

In [9]:
np.random.rand(1,)

array([0.28619978])